# gsplat :: Fit a colmap capture :: custom data

- Conda env : [gsplat](README.md#setup-a-conda-environment)

In [1]:
!nvidia-smi

Mon Sep 29 14:20:12 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 575.57.08              Driver Version: 575.57.08      CUDA Version: 12.9     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2080 Ti     On  |   00000000:01:00.0  On |                  N/A |
| 25%   45C    P5             35W /  250W |    2342MiB /  11264MiB |     28%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## Download data

In [14]:
import gdown

id = "1v_r7gjiomJNwemsXKaaJNpcTCFKl-Nbj"
vid_path = "./temp_data/waikiki_statue.mov"

gdown.download(id=id, output = vid_path)

Downloading...
From: https://drive.google.com/uc?id=1v_r7gjiomJNwemsXKaaJNpcTCFKl-Nbj
To: /home/hyunjae/110_HyunJae_Git/2025_Playgrounds/CV_Playgrounds/3DCG/gsplat/temp_data/waikiki_statue.mov
100%|██████████| 44.5M/44.5M [00:05<00:00, 7.94MB/s]


'./temp_data/waikiki_statue.mov'

In [1]:
!python ./utils/video2imgs.py --video_path "./temp_data/waikiki_statue.mov" --output_dir "./temp_data/waikiki_statue_seq/images" --fps 3 --flip -1


Saved 113 frames at 3.0 fps to './temp_data/waikiki_statue_seq/images'.


In [2]:
# !python ./utils/imgs2poses.py --data_dir "./temp_data/waikiki_statue" --factor 1
!python ./utils/imgs2poses.py --data_dir "./temp_data/waikiki_statue_seq" --factor 4 --match_type sequential_matcher

Need to run COLMAP
Features extracted
Features matched
Sparse map created
Finished running COLMAP, see ./temp_data/waikiki_statue_seq/colmap_output.txt for logs
Post-colmap
Cameras 5
Images # 113
Points (38755, 3) Visibility (38755, 113)
Depth stats 0.2749271419455869 115.51426080911165 10.197497392295682
Factors: [4]
Minifying 4 ./temp_data/waikiki_statue_seq
mogrify -resize 25% -format png *.jpg
Removed duplicates
Done
Done with imgs2poses


## 3DGS Default mode

In [3]:
!CUDA_VISIBLE_DEVICES=0 python ./temp_gsplat/examples/simple_trainer.py default \
    --data_dir ./temp_data/waikiki_statue_seq/ --data_factor 4\
    --result_dir ./temp_results/waikiki_statue_seq_default_factor_4 --save_ply --disable_viewer

[Parser] 113 images, taken by 1 cameras.
Scene scale: 1.3981006186553433
Model initialized. Number of GS: 38755
loss=0.099| sh degree=0| :   2%|▏          | 599/30000 [00:09<04:41, 104.49it/s]Step 600: 4381 GSs duplicated, 4638 GSs split. Now having 47774 GSs.
Step 600: 595 GSs pruned. Now having 47179 GSs.
loss=0.113| sh degree=0| :   2%|▎           | 699/30000 [00:10<06:32, 74.61it/s]Step 700: 7707 GSs duplicated, 8446 GSs split. Now having 63332 GSs.
Step 700: 173 GSs pruned. Now having 63159 GSs.
loss=0.099| sh degree=0| :   3%|▎          | 793/30000 [00:11<04:28, 108.89it/s]Step 800: 9168 GSs duplicated, 12366 GSs split. Now having 84693 GSs.
Step 800: 129 GSs pruned. Now having 84564 GSs.
loss=0.061| sh degree=0| :   3%|▎          | 899/30000 [00:12<04:33, 106.58it/s]Step 900: 11358 GSs duplicated, 16790 GSs split. Now having 112712 GSs.
Step 900: 167 GSs pruned. Now having 112545 GSs.
loss=0.053| sh degree=1| :   3%|▎          | 998/30000 [00:13<04:44, 101.88it/s]Step 1000: 1442

### 3DGS Default mode - Viewer

In [4]:
# !CUDA_VISIBLE_DEVICES=0 python ./temp_gsplat/examples/simple_viewer.py \
#     --ckpt ./temp_results/garden_default/ckpts/ckpt_6999_rank0.pt 

## 3DGS  3DGUT mode

In [5]:
!CUDA_VISIBLE_DEVICES=0 python ./temp_gsplat/examples/simple_trainer.py mcmc --with_ut --with_eval3d  \
    --data_dir ./temp_data/waikiki_statue_seq/ --data_factor 4\
    --result_dir ./temp_results/waikiki_statue_seq_3dgut_factor_4 --save_ply --disable_viewer

[Parser] 113 images, taken by 1 cameras.
Scene scale: 1.3981006186553433
Model initialized. Number of GS: 38755
loss=0.189| sh degree=0| :   2%|▏           | 598/30000 [00:15<15:15, 32.11it/s]Step 600: Relocated 5126 GSs.
Step 600: Added 1937 GSs. Now having 40692 GSs.
loss=0.156| sh degree=0| :   2%|▎           | 697/30000 [00:18<13:56, 35.03it/s]Step 700: Relocated 1095 GSs.
Step 700: Added 2034 GSs. Now having 42726 GSs.
loss=0.148| sh degree=0| :   3%|▎           | 798/30000 [00:21<15:05, 32.26it/s]Step 800: Relocated 998 GSs.
Step 800: Added 2136 GSs. Now having 44862 GSs.
loss=0.107| sh degree=0| :   3%|▎           | 898/30000 [00:24<14:44, 32.90it/s]Step 900: Relocated 960 GSs.
Step 900: Added 2243 GSs. Now having 47105 GSs.
loss=0.093| sh degree=1| :   3%|▍           | 999/30000 [00:26<14:14, 33.92it/s]Step 1000: Relocated 928 GSs.
Step 1000: Added 2355 GSs. Now having 49460 GSs.
loss=0.104| sh degree=1| :   4%|▍          | 1099/30000 [00:29<14:24, 33.43it/s]Step 1100: Relocate

### 3DGS  3DGUT mode - Viewer

In [6]:
# !CUDA_VISIBLE_DEVICES=0 python ./temp_gsplat/examples/simple_viewer_3dgut.py \
#     --ckpt ./temp_results/garden_3dgut/ckpts/ckpt_29999_rank0.pt 

## 3DGS MCMC(Markov Chain MonteCarlo) mode

In [7]:
!CUDA_VISIBLE_DEVICES=0 python ./temp_gsplat/examples/simple_trainer.py mcmc \
    --data_dir ./temp_data/waikiki_statue_seq/ --data_factor 4\
    --result_dir ./temp_results/waikiki_statue_seq_mcmc_factor_4 --save_ply --disable_viewer

[Parser] 113 images, taken by 1 cameras.
Scene scale: 1.3981006186553433
Model initialized. Number of GS: 38755
loss=0.177| sh degree=0| :   2%|▏           | 598/30000 [00:17<15:09, 32.32it/s]Step 600: Relocated 448 GSs.
Step 600: Added 1937 GSs. Now having 40692 GSs.
loss=0.147| sh degree=0| :   2%|▎           | 698/30000 [00:19<09:53, 49.38it/s]Step 700: Relocated 187 GSs.
Step 700: Added 2034 GSs. Now having 42726 GSs.
loss=0.147| sh degree=0| :   3%|▎           | 800/30000 [00:21<14:30, 33.56it/s]Step 800: Relocated 235 GSs.
Step 800: Added 2136 GSs. Now having 44862 GSs.
loss=0.102| sh degree=0| :   3%|▎           | 900/30000 [00:24<12:52, 37.67it/s]Step 900: Relocated 333 GSs.
Step 900: Added 2243 GSs. Now having 47105 GSs.
loss=0.086| sh degree=1| :   3%|▍           | 998/30000 [00:27<15:00, 32.20it/s]Step 1000: Relocated 421 GSs.
Step 1000: Added 2355 GSs. Now having 49460 GSs.
loss=0.100| sh degree=1| :   4%|▍          | 1089/30000 [00:29<05:55, 81.24it/s]Step 1100: Relocated 

### 3DGS MCMC mode - Viewer

In [8]:
# !CUDA_VISIBLE_DEVICES=0 python ./temp_gsplat/examples/simple_viewer.py  \
#     --ckpt ./temp_results/garden_mcmc_factor_8/ckpts/ckpt_29999_rank0.pt

## 3DGS 2DGS mode

In [9]:
!CUDA_VISIBLE_DEVICES=0 python ./temp_gsplat/examples/simple_trainer_2dgs.py \
    --data_dir ./temp_data/waikiki_statue_seq/ --data_factor 4\
    --result_dir ./temp_results/waikiki_statue_seq_2dgs_factor_4 --disable_viewer

[Parser] 113 images, taken by 1 cameras.
Scene scale: 1.3981006186553433
Model initialized. Number of GS: 38755
loss=0.096| sh degree=0| :   2%|▏           | 600/30000 [00:27<22:43, 21.56it/s]Step 600: 3879 GSs duplicated, 7473 GSs split. Now having 50107 GSs.
Step 600: 5466 GSs pruned. Now having 44641 GSs.
loss=0.119| sh degree=0| :   2%|▎           | 700/30000 [00:32<22:27, 21.74it/s]Step 700: 6751 GSs duplicated, 12410 GSs split. Now having 63802 GSs.
Step 700: 1447 GSs pruned. Now having 62355 GSs.
loss=0.103| sh degree=0| :   3%|▎           | 800/30000 [00:36<23:01, 21.13it/s]Step 800: 7877 GSs duplicated, 15698 GSs split. Now having 85930 GSs.
Step 800: 3290 GSs pruned. Now having 82640 GSs.
loss=0.068| sh degree=0| :   3%|▎           | 900/30000 [00:41<22:32, 21.52it/s]Step 900: 8684 GSs duplicated, 19693 GSs split. Now having 111017 GSs.
Step 900: 4002 GSs pruned. Now having 107015 GSs.
loss=0.058| sh degree=1| :   3%|▎          | 1000/30000 [00:45<22:31, 21.45it/s]Step 1000: 

### 3DGS  2DGS mode - Viewer

In [10]:
# !CUDA_VISIBLE_DEVICES=0 python ./temp_gsplat/examples/simple_viewer_2dgs.py \
#     --ckpt ./temp_results/garden_2dgs/ckpts/ckpt_29999.pt